# Remove Radiology Report

The goal here is to use meds_tools to filter df_patient, remove notes in the 'table' column, and rematerialize csvs without the report

In [1]:
import os
import argparse
import json
import yaml
import re
import pandas as pd
from pathlib import Path
from torch.utils.data import DataLoader
from google.cloud import bigquery
from google.cloud import storage
import meds_reader
from meds_tools import patient_timeline
from meds2text.ontology import OntologyDescriptionLookupTable

/home/dcunhrya/vista_eval/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
path = '/home/dcunhrya/vista_bench/radiation_treatment_outcomes_v1_1/radiation_outcome_subsampled.csv'
df = pd.read_csv(path)

In [6]:
df['patient_string'].head(1).tolist()

['[1959-04-23 00:00] | MEDS_BIRTH\n[1959-04-23 00:00] | MEDS_BIRTH\n[1959-04-23 00:00] | Ethnicity/38003564\n[1959-04-23 00:00] | Race/8515\n[1959-04-23 00:00] | Gender/8532\n[2006-02-12 00:00] | ICD9CM/493.90 (Asthma, unspecified type, unspecified)\n[2012-01-08 00:00] | ICD9CM/795.51 (Nonspecific reaction to tuberculin skin test without active tuberculosis)\n[2012-11-01 18:45] | STANFORD_PROC/US ABDOMEN LIMITED (US ABDOMEN LIMITED)\n[2013-01-22 14:37] | STANFORD_NOTE/or-preop | NOTE: Formatting of this note is different from the original.  PreOperative Telephone History    Procedure Information:  Pre-Op Diagnosis Codes:     * BILIARY DYSKINESIA [575.8]  Procedure(s) (LRB):  CHOLECYSTECTOMY LAPAROSCOPIC (N/A)  Date of Procedure:  12/22/2012  Surgeon:  Kujawa, Rylan Sledge (M.D.)    Venue:  STR-AMBULATORY-OR  Requested Anesthetic type:  General Anesthesia    History provided by: Patient:  Mazie M Chan    Chief Complaint and History of Present Illness:  Neff is a 53 Y female is scheduled